# Вычисления на GPU

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы: 
* Макрушин С.В. Лекция "GPGPU"
* https://numpy.org/doc/stable/reference/generated/numpy.load.html
* https://docs.cupy.dev/en/stable/user_guide/basic.html
* https://carpentries-incubator.github.io/gpu-speedups/01_CuPy_and_Numba_on_the_GPU/index.html
* https://carpentries-incubator.github.io/lesson-gpu-programming/02-cupy/index.html
* https://carpentries-incubator.github.io/gpu-speedups/01_CuPy_and_Numba_on_the_GPU/index.html
* https://www.youtube.com/watch?v=9bBsvpg-Xlk
* https://numba.pydata.org/numba-doc/latest/user/vectorize.html
* https://numba.pydata.org/numba-doc/latest/cuda/ufunc.html
* https://towardsdatascience.com/cuda-by-numba-examples-1-4-e0d06651612f


Для выполнения данной работы рекомендуется использовать среду Google Colab. Перед началом работы смените среду выполнения на GPU. 
Обратите внимание, что время графического процессора, которое предоставляет Google, ограничено. 

In [ ]:
!nvcc --version

In [ ]:
!nvidia-smi

## Задачи для совместного разбора

1. Создайте массив `cupy`, заполненный числами из стандартного нормального распределения. Исследуйте основные характеристики массива (размеры, устройство, тип). Сравните время выполнения нескольких аналогичных операций с `numpy` и `cupy`. 

2. Напишите функцию `f(x, y)`, которая считает sin(x) + cos(y), если они одинакового знака (или хотя бы одно из них равно нулю), и cos(x) + sin(y) в противном случае. Векторизуйте функцию при помощи numba с различными значениями аргумента `target`. Создайте 2 массива и выполните вычисления. Создайте ядро CUDA и выполните те же вычисления.

![image.png](attachment:image.png)

3. Создайте два тензора `A` и `B`, заполненные числами из стандартного нормального распределения. Решите матричное уравнение `AX = B`. Повторите решение с использованием GPU.

## Лабораторная работа 15

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy`, `cupy`, `numba` и `torch`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import cupy as cp

### CyPy

<p class="task" id="1"></p>

1\. В файле "blobs.npz" находится четыре массива: 
* X - 1 млн 128-мерных векторов;
* centers - центроиды кластеров;
* y1 - вариант кластеризации точек;
* y2 - альтернативный вариант кластеризации точек.

Считайте эти данные в виде массива `cupy.ndarray`. Используя пакет `cupy`, вычислите и выведите на экран:
* размерость каждого из заданных массивов;
* объем памяти, занимаемой каждым из массивов (в мегабайтах);
* количество точек в каждом из кластеров в соответствии с разбиением `y1`;
* количество точек в каждом из кластеров в соответствии с разбиением `y2`.

In [3]:
%%time
data_cp = cp.load('/content/drive/My Drive/Colab Notebooks/blobs.npz')
X_cp, y1_cp, y2_cp, centers_cp = data_cp['X'], data_cp['y1'], data_cp['y2'], data_cp['centers']

CPU times: user 1.54 s, sys: 1.18 s, total: 2.72 s
Wall time: 14.8 s


In [4]:
X_cp.shape, y1_cp.shape, y2_cp.shape, centers_cp.shape

((1000000, 128), (1000000,), (1000000,), (3, 128))

In [6]:
X_cp.nbytes / 1024**2, y1_cp.nbytes / 1024**2, y2_cp.nbytes / 1024**2, centers_cp.nbytes / 1024**2

(976.5625, 7.62939453125, 7.62939453125, 0.0029296875)

In [8]:
cp.asarray(cp.unique(y1_cp, return_counts=True)).T

array([[     0, 333885],
       [     1, 333352],
       [     2, 332763]])

In [12]:
cp.asarray(cp.unique(y2_cp, return_counts=True)).T

array([[     0, 333334],
       [     1, 333333],
       [     2, 333333]])

<p class="task" id="2"></p>

2\. Используя пакет `numpy`, выясните, какое из разбиений (`y1` или `y2`) лучше разделяет данные на кластеры. Для этого вычислите среднее значение евклидова расстояния от каждой точки до центроида присвоенного ей кластера. Выведите данные значения для обоих разбиений на экран и сделайте вывод. Замерьте время выполнения решения при помощи магической команды `timeit`.

In [16]:
data = np.load('/content/drive/My Drive/Colab Notebooks/blobs.npz')

In [17]:
X_np, y1_np, y2_np, centers_np = data['X'], data['y1'], data['y2'], data['centers']

In [18]:
%%timeit
avg_dist_y1_np = np.mean(np.sqrt(np.sum((X_np - centers_np[y1_np])**2,axis=1)))
avg_dist_y2_np = np.mean(np.sqrt(np.sum((X_np - centers_np[y2_np])**2,axis=1)))

1.67 s ± 42.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
avg_dist_y1_np = np.mean(np.sqrt(np.sum((X_np - centers_np[y1_np])**2,axis=1)))
avg_dist_y2_np = np.mean(np.sqrt(np.sum((X_np - centers_np[y2_np])**2,axis=1)))
avg_dist_y1_np, avg_dist_y2_np

(69.6272457739018, 11.292487638331732)

<p class="task" id="3"></p>

3\. Решите задачу 2, воспользовавшись пакетом `cupy`. Замерьте время выполнения решения при помощи магической команды `timeit`. Сравните время выполнения двух решений. Подтвердите (программно), что результаты решений совпадают.

In [13]:
%%timeit
avg_dist_y1_cp = cp.mean(cp.sqrt(cp.sum((X_cp - centers_cp[y1_cp])**2,axis=1)))
avg_dist_y2_cp = cp.mean(cp.sqrt(cp.sum((X_cp - centers_cp[y2_cp])**2,axis=1)))

457 µs ± 36.1 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
avg_dist_y1_cp = cp.mean(cp.sqrt(cp.sum((X_cp - centers_cp[y1_cp])**2,axis=1)))
avg_dist_y2_cp = cp.mean(cp.sqrt(cp.sum((X_cp - centers_cp[y2_cp])**2,axis=1)))
avg_dist_y1_cp, avg_dist_y2_cp

(array(69.62724577), array(11.29248764))

In [23]:
avg_dist_y1_cp == avg_dist_y1_np

array(True)

In [24]:
avg_dist_y2_cp == avg_dist_y2_np

array(False)

<p class="task" id="4"></p>

4\. Напишите функцию `test_y`, которая считает среднее значение евклидова расстояния от каждой точки до центроида присвоенного ей кластера. Реализуйте функцию таким образом, чтобы она могла корректно работать и с массивами `numpy`, и с массивами `cupy`. Если в функции передается часть аргументов в виде массивов `numpy`, а часть - в виде массивов `cupy`, должно быть возбуждено исключение `ValueError`. Подтвердите правильность решения различными примерами.

```python
def test_y(X, y, centers):
  pass
```

In [47]:
def test_y(X, y, centers):
  if all(isinstance(n, cp.ndarray) for n in [X, y, centers]):
    return cp.mean(cp.sqrt(cp.sum((X_cp - centers_cp[y1_cp])**2,axis=1)))
  elif all(isinstance(n, np.ndarray) for n in [X, y, centers]):
    return np.mean(np.sqrt(np.sum((X_np - centers_np[y1_np])**2,axis=1)))
  else:
    raise ValueError

In [48]:
test_y(X_cp, y1_cp, centers_cp)

array(69.62724577)

In [49]:
test_y(X_np, y1_np, centers_np)

69.6272457739018

In [50]:
test_y(X_cp, y1_np, centers_np)

ValueError: ignored

### Numba

<p class="task" id="5"></p>

5\. Даны массива `numbers` и `powers`. Реализуйте функцию `power_sum`, которая находит сумму всех цифр числа `number`, возведенных в степень `power`. Для каждой пары `(numbers[i], power[i])` вычислите `power_sum` , найдите максимум этой величины и выведите ее на экран. Решите задачу при помощи декоратора `numba.vectorize` с различными значениями аргумента `target`: `cpu`, `parallel` и `cuda`. Замерьте время выполнения каждого из решений.

```python
def power_sum(number: int, power: int) -> int:
  pass
```

In [63]:
import numba

In [64]:
import numpy as np
np.random.seed(42)

N = 10_000_000

numbers = np.random.randint(0, 1000, size=N)
powers = np.random.randint(1, 5, size=N)

In [61]:
def power_sum(number: int, power: int) -> int:
  s = 0
  while number > 0:
    s += (number%10) ** power
    number //= 10
  return s

In [65]:
power_sum_cpu = numba.vectorize(["int64(int64, int64)"], target="cpu")(power_sum)
power_sum_parallel = numba.vectorize(["int64(int64, int64)"], target="parallel")(power_sum)
power_sum_cuda = numba.vectorize(["int64(int64, int64)"], target="cuda")(power_sum)

In [201]:
%%time
power_sum_cpu(numbers, powers).max()

CPU times: user 168 ms, sys: 24.1 ms, total: 192 ms
Wall time: 191 ms


19683

In [202]:
%%time
power_sum_parallel(numbers, powers).max()

CPU times: user 456 ms, sys: 23.9 ms, total: 480 ms
Wall time: 430 ms


19683

In [203]:
%%time
power_sum_cuda(numbers, powers).max()

CPU times: user 68.8 ms, sys: 27.2 ms, total: 96 ms
Wall time: 102 ms


19683

<p class="task" id="6"></p>

6\. Преобразуйте `power_sum` в device function при помощи декоратора `numba.cuda.jit`. Напишите ядро `power_sum_kernel` и решите с его помощью задачу 5.

In [72]:
from numba import cuda

In [73]:
@cuda.jit(device=True)
def power_sum_device(number: int, power: int) -> int:
  s = 0
  while number > 0:
    s += (number%10) ** power
    number //= 10
  return s

In [75]:
@cuda.jit
def power_sum_kernel(numbers, powers, res):
  i = cuda.grid(1)
  res[i] = power_sum_device(numbers[i], powers[i])

In [81]:
res = np.zeros(N)
threadsperblock = 512
blockspergrid = (N + threadsperblock - 1) // threadsperblock
blockspergrid, res

(19532, array([0., 0., 0., ..., 0., 0., 0.]))

In [82]:
19532 * 512 > N

True

In [205]:
%%time
power_sum_kernel[blockspergrid, threadsperblock](numbers, powers, res)
res.max()

CPU times: user 119 ms, sys: 1.43 ms, total: 121 ms
Wall time: 120 ms


/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


19683.0

In [91]:
res

array([9.000e+00, 2.160e+02, 5.392e+03, ..., 1.000e+00, 1.770e+02,
       1.340e+02])

In [206]:
all(res == power_sum_parallel(numbers, powers))

True

In [90]:
len(res)

10000000

### PyTorch

<p class="task" id="7"></p>

7\. Создайте тензор размера 256х64. Заполните его числами из равномерного распределения $U(-a, a)$, где $a=3\sqrt{\frac{6}{256+64}}$. Выведите на экран основные характеристики тензора:
* кол-во строк;
* кол-во столбцов;
* кол-во элементов;
* тип данных;
* устройство.

In [92]:
import torch as th

In [127]:
a = 3 * np.sqrt(6/(256+64))
U = (a-(-a)) * th.rand(size=(256, 64)) + (-a)
U

tensor([[ 2.2572e-01,  6.3576e-02,  8.5264e-05,  ..., -3.6757e-01,
          2.8168e-01,  1.1565e-01],
        [ 6.8596e-02, -1.9135e-01, -3.4950e-03,  ..., -1.2086e-01,
         -8.5516e-02,  2.6054e-01],
        [-1.8674e-01, -2.1401e-01,  2.3157e-02,  ..., -1.2209e-01,
         -1.6391e-01, -3.4237e-01],
        ...,
        [-7.0165e-02,  1.4363e-01,  2.2325e-01,  ...,  4.2570e-02,
         -1.0668e-02,  1.1390e-01],
        [-2.2553e-01, -2.3075e-01, -6.1607e-02,  ..., -2.2642e-01,
          2.2104e-01, -8.4577e-02],
        [ 2.9704e-01, -3.0796e-01,  7.9771e-02,  ..., -5.9742e-02,
         -2.5226e-02, -1.8075e-01]])

In [128]:
U.shape

torch.Size([256, 64])

In [129]:
U.numel()

16384

In [130]:
U.dtype

torch.float32

In [131]:
U.device

device(type='cpu')

<p class="task" id="8"></p>

8\. Создайте тензор `A`, заполненный целыми числами из отрезка [-1, 1] (при создании зафиксируйте зерно ГПСЧ, равное 42). Получите матрицу B по правилу $B = \frac{A + A^T}{2}$. Проверьте, является ли матрица симметричной. Используя `torch`, получите SVD-разложение матрицы `B`. Проверьте, насколько точно разложение приближает исходную матрицу (для оценки качества используйте норму Фробениуса). Измерьте время выполнения решения.

In [186]:
th.manual_seed(42)
th.random.initial_seed()

42

In [191]:
A = th.randint(-1,1,(64,64))
B = (A + A.T) / 2

A, B

(tensor([[-1,  0,  0,  ...,  0,  0,  0],
         [-1, -1,  0,  ...,  0, -1, -1],
         [-1, -1,  0,  ..., -1,  0, -1],
         ...,
         [-1, -1, -1,  ...,  0,  0, -1],
         [-1, -1,  0,  ..., -1, -1, -1],
         [ 0,  0,  0,  ...,  0, -1, -1]]),
 tensor([[-1.0000, -0.5000, -0.5000,  ..., -0.5000, -0.5000,  0.0000],
         [-0.5000, -1.0000, -0.5000,  ..., -0.5000, -1.0000, -0.5000],
         [-0.5000, -0.5000,  0.0000,  ..., -1.0000,  0.0000, -0.5000],
         ...,
         [-0.5000, -0.5000, -1.0000,  ...,  0.0000, -0.5000, -0.5000],
         [-0.5000, -1.0000,  0.0000,  ..., -0.5000, -1.0000, -1.0000],
         [ 0.0000, -0.5000, -0.5000,  ..., -0.5000, -1.0000, -1.0000]]))

In [192]:
(B == B.T).all()

tensor(True)

In [189]:
U, S, Vh = th.linalg.svd(B)
th.sqrt(((U@th.diag(S)@Vh - B)**2).sum())

tensor(2.4836e-05)

In [193]:
%%time
A = th.randint(-1,1,(64,64))
B = (A + A.T) / 2
B == B.T
U, S, Vh = th.linalg.svd(B)
th.sqrt(((U@th.diag(S)@Vh - B)**2).sum())

CPU times: user 2.2 ms, sys: 38 µs, total: 2.24 ms
Wall time: 2.1 ms


tensor(2.4976e-05)

<p class="task" id="9"></p>

9\. Повторите решение задачи 8, перенеся вычисления на GPU. Сравните результаты и время выполнения двух решений.

In [198]:
%%time
A_gpu = A.cuda()
B_gpu = (A_gpu + A_gpu.T) / 2
B_gpu == B_gpu.T
U_gpu, S_gpu, Vh_gpu = th.linalg.svd(B_gpu)
th.sqrt(((U_gpu@th.diag(S_gpu)@Vh_gpu - B_gpu)**2).sum())

CPU times: user 4.17 ms, sys: 6 µs, total: 4.17 ms
Wall time: 4.18 ms


tensor(0.0004, device='cuda:0')

In [200]:
4.18 / 2.1

1.9904761904761903

In [199]:
0.0004/2.4976e-05

16.01537475976938

**Вычисления на GPU выполнились в 2 раза дольше.\
Результаты отличаются в 16 раз.**